In [11]:
import os, sys
from tqdm import tqdm
from sqlitedict import SqliteDict

from wiki_database import WikiDatabaseSqlite, Text
from utils_doc_results_db import get_empty_tag_dict, get_tag_2_id_dict, get_tag_dict, get_vocab_tf_idf_from_exp, get_tf_idf_name, get_vocab_tf_idf_from_exp
from utils_doc_results_db import get_dict_from_n_gram
from utils_db import dict_load_json, dict_save_json, HiddenPrints, load_jsonl
from utils_doc_results import Claim, ClaimDocTokenizer, get_tag_word_from_wordtag

import config

In [12]:
# === variables === #
n_gram = 1
claim_data_set = 'dev'
folder_name_score_combination = 'score_combination'

path_dir_results = os.path.join(config.ROOT, config.RESULTS_DIR, folder_name_score_combination)
path_tags = os.path.join(path_dir_results, 'tags_' + claim_data_set + '_n_gram_' + str(n_gram) + '.json')
# path_dir_files = os.path.join(path_dir_results, claim_data_set)
path_dir_claims = os.path.join(path_dir_results, 'claims_' + claim_data_set)
path_wiki_pages = os.path.join(config.ROOT, config.DATA_DIR, config.WIKI_PAGES_DIR, 'wiki-pages')
path_wiki_database_dir = os.path.join(config.ROOT, config.DATA_DIR, config.DATABASE_DIR)
path_claim_data_set = os.path.join(config.ROOT, config.DATA_DIR, config.RAW_DATA_DIR, claim_data_set + ".jsonl")

try:
    os.makedirs(path_dir_results, exist_ok=True)
except FileExistsError:
    print('folder already exists:', path_dir_results)

try:
    os.makedirs(path_dir_claims, exist_ok=True)
except FileExistsError:
    print('folder already exists:', path_dir_claim)

results = load_jsonl(path_claim_data_set)
wiki_database = WikiDatabaseSqlite(path_wiki_database_dir, path_wiki_pages)
tag_2_id_dict = get_tag_2_id_dict()

tag_dict = get_tag_dict(claim_data_set, n_gram, path_tags, wiki_database)
nr_claims = 20#len(results)

Load existing settings file
Load title dictionary
tags file already exists


In [ ]:
print('claim database: insert claim\'s text and claim\'s tag_list')

for str_id, tag_list in tqdm(tag_dict.items(), total = len(tag_dict), desc = 'tag'):
    id = int(str_id)
    if id < nr_claims:
        file = ClaimFile(id = id, path_dir_files = path_dir_claims)
        file.process_tags(tag_list, n_gram)
        file.process_claim(results[id])

print('claim database: insert nr words per tag for claim')

experiment_nr = 37
with HiddenPrints():
    vocab, tf_idf_db = get_vocab_tf_idf_from_exp(experiment_nr, wiki_database)

for id in tqdm(range(nr_claims), desc = 'nr words per pos'):
    file = ClaimFile(id = id, path_dir_files = path_dir_claims)
    file.process_nr_words_per_pos(tf_idf_db, tag_2_id_dict)

print('claim database: ')

In [26]:
from sqlitedict import SqliteDict
from doc_results_db_utils import get_vocab_tf_idf_from_exp
from utils_db import HiddenPrints

# === run experiment === #
experiment_nr = 31

print('load tf_idf nr_words_pos')
with HiddenPrints():
    vocab, tf_idf_db = get_vocab_tf_idf_from_exp(experiment_nr, wiki_database)

mydict_ids = SqliteDict(tf_idf_db.path_ids_dict)
mydict_tf_idf = SqliteDict(tf_idf_db.path_tf_idf_dict)



load tf_idf nr_words_pos


In [30]:
from tqdm import tqdm

nr_claims_selected = 1000 #nr_claims

for id in tqdm(range(nr_claims_selected), desc = 'nr words per pos'):
    file = ClaimFile(id = id, path_dir_files = path_dir_claims)
    file.process_tf_idf_experiment(tag_2_id_dict, tf_idf_db, mydict_ids, mydict_tf_idf)
    




nr words per pos:   0%|          | 0/1000 [00:00<?, ?it/s]

nr words per pos:   0%|          | 1/1000 [00:14<3:56:22, 14.20s/it]

nr words per pos:   0%|          | 2/1000 [00:14<2:48:32, 10.13s/it]

nr words per pos:   0%|          | 3/1000 [00:41<4:10:56, 15.10s/it]

nr words per pos:   0%|          | 4/1000 [00:46<3:22:02, 12.17s/it]

nr words per pos:   0%|          | 5/1000 [00:53<2:52:36, 10.41s/it]

nr words per pos:   1%|          | 6/1000 [00:55<2:11:49,  7.96s/it]

nr words per pos:   1%|          | 7/1000 [01:01<2:04:08,  7.50s/it]

nr words per pos:   1%|          | 8/1000 [01:02<1:28:43,  5.37s/it]

nr words per pos:   1%|          | 9/1000 [01:09<1:40:11,  6.07s/it]

nr words per pos:   1%|          | 10/1000 [01:10<1:12:58,  4.42s/it]

nr words per pos:   1%|          | 11/1000 [01:15<1:14:50,  4.54s/it]

nr words per pos:   1%|          | 12/1000 [01:15<53:59,  3.28s/it]  

nr words per pos:   1%|▏         | 13/1000 [01:15<39:19,  2.39s/it]

nr words per pos:   1%|▏  

nr words per pos:  21%|██        | 212/1000 [24:30<1:19:34,  6.06s/it]

nr words per pos:  21%|██▏       | 213/1000 [24:33<1:08:04,  5.19s/it]

nr words per pos:  21%|██▏       | 214/1000 [24:39<1:09:47,  5.33s/it]

nr words per pos:  22%|██▏       | 215/1000 [24:44<1:09:01,  5.28s/it]

nr words per pos:  22%|██▏       | 216/1000 [24:47<1:01:08,  4.68s/it]

nr words per pos:  22%|██▏       | 217/1000 [24:55<1:12:50,  5.58s/it]

nr words per pos:  22%|██▏       | 218/1000 [25:04<1:23:57,  6.44s/it]

nr words per pos:  22%|██▏       | 219/1000 [25:07<1:10:26,  5.41s/it]

nr words per pos:  22%|██▏       | 220/1000 [25:23<1:51:59,  8.61s/it]

nr words per pos:  22%|██▏       | 221/1000 [25:25<1:27:14,  6.72s/it]

nr words per pos:  22%|██▏       | 222/1000 [25:32<1:28:53,  6.86s/it]

nr words per pos:  22%|██▏       | 223/1000 [25:43<1:42:40,  7.93s/it]

nr words per pos:  22%|██▏       | 224/1000 [25:46<1:23:39,  6.47s/it]

nr words per pos:  22%|██▎       | 225/1000 [25:51<1:20:52,  6.2

nr words per pos:  42%|████▎     | 425/1000 [43:48<48:10,  5.03s/it]

nr words per pos:  43%|████▎     | 426/1000 [43:49<37:17,  3.90s/it]

nr words per pos:  43%|████▎     | 427/1000 [43:52<33:47,  3.54s/it]

nr words per pos:  43%|████▎     | 428/1000 [43:55<32:09,  3.37s/it]

nr words per pos:  43%|████▎     | 429/1000 [43:56<26:40,  2.80s/it]

nr words per pos:  43%|████▎     | 430/1000 [43:58<23:20,  2.46s/it]

nr words per pos:  43%|████▎     | 431/1000 [43:59<21:00,  2.21s/it]

nr words per pos:  43%|████▎     | 432/1000 [44:06<31:55,  3.37s/it]

nr words per pos:  43%|████▎     | 433/1000 [44:08<29:42,  3.14s/it]

nr words per pos:  43%|████▎     | 434/1000 [44:11<30:04,  3.19s/it]

nr words per pos:  44%|████▎     | 435/1000 [44:15<30:54,  3.28s/it]

nr words per pos:  44%|████▎     | 436/1000 [44:16<23:26,  2.49s/it]

nr words per pos:  44%|████▎     | 437/1000 [44:17<20:53,  2.23s/it]

nr words per pos:  44%|████▍     | 438/1000 [44:19<19:42,  2.10s/it]

nr words per pos:  4

nr words per pos:  64%|██████▍   | 644/1000 [59:04<26:46,  4.51s/it]

nr words per pos:  64%|██████▍   | 645/1000 [59:10<30:07,  5.09s/it]

nr words per pos:  65%|██████▍   | 646/1000 [59:25<47:29,  8.05s/it]

nr words per pos:  65%|██████▍   | 647/1000 [59:26<35:03,  5.96s/it]

nr words per pos:  65%|██████▍   | 648/1000 [59:35<40:15,  6.86s/it]

nr words per pos:  65%|██████▍   | 649/1000 [59:40<35:44,  6.11s/it]

nr words per pos:  65%|██████▌   | 650/1000 [59:40<25:30,  4.37s/it]

nr words per pos:  65%|██████▌   | 652/1000 [1:00:01<36:17,  6.26s/it]

nr words per pos:  65%|██████▌   | 653/1000 [1:00:05<31:05,  5.38s/it]

nr words per pos:  65%|██████▌   | 654/1000 [1:00:11<33:35,  5.83s/it]

nr words per pos:  66%|██████▌   | 655/1000 [1:00:14<27:05,  4.71s/it]

nr words per pos:  66%|██████▌   | 656/1000 [1:00:21<31:18,  5.46s/it]

nr words per pos:  66%|██████▌   | 657/1000 [1:00:25<28:19,  4.95s/it]

nr words per pos:  66%|██████▌   | 658/1000 [1:00:28<25:18,  4.44s/it]

nr wor

nr words per pos:  85%|████████▌ | 853/1000 [16:13:18<06:28,  2.64s/it]

nr words per pos:  85%|████████▌ | 854/1000 [16:13:21<06:21,  2.62s/it]

nr words per pos:  86%|████████▌ | 855/1000 [16:13:25<07:20,  3.04s/it]

nr words per pos:  86%|████████▌ | 856/1000 [16:13:28<07:24,  3.09s/it]

nr words per pos:  86%|████████▌ | 857/1000 [16:13:33<08:28,  3.56s/it]

nr words per pos:  86%|████████▌ | 858/1000 [16:13:37<08:40,  3.66s/it]

nr words per pos:  86%|████████▌ | 859/1000 [16:13:41<09:12,  3.92s/it]

nr words per pos:  86%|████████▌ | 860/1000 [16:13:42<06:54,  2.96s/it]

nr words per pos:  86%|████████▌ | 861/1000 [16:13:42<05:04,  2.19s/it]

nr words per pos:  86%|████████▌ | 862/1000 [16:13:50<08:36,  3.75s/it]

nr words per pos:  86%|████████▋ | 863/1000 [16:14:03<15:25,  6.76s/it]

nr words per pos:  86%|████████▋ | 864/1000 [16:14:05<11:58,  5.28s/it]

nr words per pos:  86%|████████▋ | 865/1000 [16:14:13<13:48,  6.13s/it]

nr words per pos:  87%|████████▋ | 866/1000 [16:14:

In [57]:
from utils_db import load_jsonl

path_results = '/home/bmelman/Desktop/C_disk/02_university/06_thesis/01_code/fever/_04_results/vocab_title_1_lp/thr_0.01_/ex_term_frequency_inverse_document_frequency_title/01_results/predicted_labels_20.json'
dict_results = load_jsonl(path_results)
claim_nr = 9
interpreter_list = dict_results[claim_nr]['evidence']
print(interpreter_list)

id_list = []
for interpreter in interpreter_list:
    for proof in interpreter:
        title = proof[2]
        if title is not None:
            id = wiki_database.get_id_from_title(title)
            id_list.append(id)


[[[127089, 141573, 'Andrew_Kevin_Walker', 0]]]


In [97]:
from utils_db import load_jsonl

path_results = '/home/bmelman/Desktop/C_disk/02_university/06_thesis/01_code/fever/_04_results/vocab_title_1_lp/thr_0.01_/ex_term_frequency_inverse_document_frequency_title/01_results/predicted_labels_20.json'
dict_results = load_jsonl(path_results)
claim_nr = 9
interpreter_list = dict_results[claim_nr]
interpreter_list

{'id': 108281,
 'verifiable': 'VERIFIABLE',
 'label': 'REFUTES',
 'claim': 'Andrew Kevin Walker is only Chinese.',
 'evidence': [[[127089, 141573, 'Andrew_Kevin_Walker', 0]]],
 'docs_selected': [3680518,
  3680749,
  3682302,
  4041240,
  4323813,
  4548726,
  4566445,
  4617659,
  4724012,
  2611161,
  2703062,
  531639,
  480771,
  484565,
  465538,
  2666840,
  2654995,
  5182741,
  1001990,
  3614807],
 'results': {'e_score': 1.0, 'f_score': 1, 'f_score_labelled': 1}}

In [100]:
from utils_db import dict_save_json, dict_load_json
path_dict = 'tmp.json'
dict = {}
dict['Simón_Bolívar'] = 1
dict_save_json(dict, path_dict)
dict['Simón_Bolívar']

overwriting file: tmp.json


1

In [78]:
with SqliteDict('tmp.sqlite') as dict_tmp:
    dict_tmp['Simón_Bolívar'] = 1
    dict_tmp.commit()
with SqliteDict('tmp.sqlite') as dict_tmp:
    print(dict_tmp['Simón_Bolívar'])

In [30]:
import unicodedata
unicodedata.normalize('NFD', 'Simón_Bolívar')

'Simón_Bolívar'

In [ ]:
list_titles = list(wiki_database.title_2_id_db.keys())

In [17]:
from tqdm import tqdm
id_list = []
for key, value in tqdm(wiki_database.title_2_id_db.items()):
    if 'n_Bol' in key:
        id_list.append(key)

5416536it [01:13, 74033.34it/s]


In [32]:
start = 460
unicodedata.normalize('NFD', id_list[460]) == unicodedata.normalize('NFD', 'Simón_Bolívar')


True

In [95]:
from utils_db import load_jsonl

path_text = '/home/bmelman/Desktop/C_disk/02_university/06_thesis/01_code/fever/_01_data/_02_wikipedia_pages/wiki-pages/wiki-090.jsonl'

text_list = load_jsonl(path_text)
id = 2001
print(text_list[id])
print(text_list[id])

{'id': 'St._Francis_de_Sales_High_School_-LRB-Detroit,_Michigan-RRB-', 'text': 'St. Francis de Sales High School was a coeducational Catholic high school in Detroit , Michigan . The school closed in 1971 . Since 1994 , the school building has been home to Loyola High School . ', 'lines': '0\tSt. Francis de Sales High School was a coeducational Catholic high school in Detroit , Michigan .\tDetroit\tDetroit\tMichigan\tMichigan\tCatholic\tCatholic\n1\tThe school closed in 1971 .\n2\tSince 1994 , the school building has been home to Loyola High School .\tLoyola High School\tLoyola High School (Detroit)\n3\t'}
{'id': 'St._Francis_de_Sales_High_School_-LRB-Detroit,_Michigan-RRB-', 'text': 'St. Francis de Sales High School was a coeducational Catholic high school in Detroit , Michigan . The school closed in 1971 . Since 1994 , the school building has been home to Loyola High School . ', 'lines': '0\tSt. Francis de Sales High School was a coeducational Catholic high school in Detroit , Michiga

In [177]:
claim_db_simon = claim_db.get_claim_from_id(26)['evidence'][0][0][2]
# wiki_db_simon = unicodedata.normalize('NFD', normalise_text(id_list[460]))
# claim_db_simon == normalise_text(id_list[460])

In [183]:
with SqliteDict('tmp.sqlite') as dict_tmp:
    dict_tmp[normalise_text(id_list[460])] = normalise_text(id_list[460])
    dict_tmp.commit()
with SqliteDict('tmp.sqlite') as dict_tmp:
    print(dict_tmp[claim_db_simon])
    print(dict_tmp[claim_db_simon] == claim_db_simon)

Simón Bolívar
True


In [156]:
from tqdm import tqdm
id_list = []
for key, value in tqdm(wiki_database.title_2_id_db.items()):
    if 'n_Bol' in key:
        id_list.append(key)

5416536it [02:07, 42616.98it/s]


In [158]:
id_list[460]

'Simón_Bolívar'

In [149]:
import config 

claim_data_set = 'dev'
path_dir_database = os.path.join(config.ROOT, config.DATA_DIR, config.DATABASE_DIR)
path_raw_data = os.path.join(config.ROOT, config.DATA_DIR, config.RAW_DATA_DIR)

claim_db = ClaimDatabase(path_dir_database = path_dir_database, path_raw_data = path_raw_data, claim_data_set = claim_data_set)

for i in tnrange(1000):
    claim_db.get_claim_from_id(i)

claim database already exists


In [139]:
import os
import unicodedata
from utils_db import dict_load_json, dict_save_json, load_jsonl
from tqdm import tqdm

class ClaimDatabase:
    def __init__(self, path_dir_database, path_raw_data, claim_data_set):
        self.path_dir_database = path_dir_database
        self.path_raw_data = path_raw_data
        self.claim_data_set = claim_data_set
        
        self.path_dir_database_claims = os.path.join(self.path_dir_database, 'claims_' + str(self.claim_data_set))
        self.path_raw_claims = os.path.join(path_raw_data, str(self.claim_data_set) + '.jsonl')
        self.path_settings = os.path.join(path_dir_database_claims, 'settings.json')
        
        if not os.path.isdir(self.path_dir_database_claims):
            print('create claim database')
            os.makedirs(self.path_dir_database_claims)
            self.create_database()
        else:
            print('claim database already exists')
        
        if os.path.isfile(self.path_settings):
            self.settings = dict_load_json(self.path_settings)
            self.nr_claims = settings['nr_claims']
        else:
            raise ValueError('settings file should exist')
        
        
    def create_database(self):
        list_claim_dicts = load_jsonl(self.path_raw_claims)
        self.nr_claims = len(list_claim_dicts)
        for id in tqdm(range(self.nr_claims)):
            path_claim = os.path.join(self.path_dir_database_claims, str(id) + '.json')
            dict_claim_id = list_claim_dicts[id]
            dict_claim_id['verifiable'] = unicodedata.normalize('NFD', normalise_text(dict_claim_id['verifiable']))
            dict_claim_id['claim'] = unicodedata.normalize('NFD', normalise_text(dict_claim_id['claim']))
            for interpreter in range(len(dict_claim_id['evidence'])):
                for proof in range(len(dict_claim_id['evidence'][interpreter])):
                    if dict_claim_id['evidence'][interpreter][proof][2] != None:
                        dict_claim_id['evidence'][interpreter][proof][2] = unicodedata.normalize('NFD', normalise_text(dict_claim_id['evidence'][interpreter][proof][2]))
            
            dict_save_json(dict_claim_id, path_claim)
        
        if os.path.isfile(self.path_settings):
            settings = dict_load_json(self.path_settings)
            
        else:
            settings = {}
        
        settings['nr_claims'] = self.nr_claims
        dict_save_json(settings, self.path_settings)
    
    def get_claim_from_id(self, id):
        path_claim = os.path.join(self.path_dir_database_claims, str(id) + '.json')
        dict_claim_id = dict_load_json(path_claim)
        return dict_claim_id
    
            
            

In [22]:
from utils_db import dict_save_json
from doc_results_db_utils import get_tf_idf_name
from wiki_database import Text
from doc_results_db import get_dict_from_n_gram
from tqdm import tnrange, tqdm
from utils_doc_results import Claim, ClaimDocTokenizer

class ClaimFile:
    """A sample Employee class"""
    def __init__(self, id, path_dir_files):
        self.path_claim = os.path.join(path_dir_files, str(id) + '.json')
        if os.path.isfile(self.path_claim):
            self.claim_dict = dict_load_json(self.path_claim)
        else:
            self.claim_dict = {}
            self.claim_dict['claim'] = {}
            self.claim_dict['claim']['1_gram'] = {}
            self.claim_dict['claim']['1_gram']['nr_words'] = None
            self.claim_dict['claim']['1_gram']['nr_words_per_pos'] = get_empty_tag_dict()
            self.claim_dict['title'] = {}
            self.claim_dict['title']['1_gram'] = {}
#             self.claim_dict['title']['1_gram']['nr_words'] = None
#             self.claim_dict['title']['1_gram']['nr_words_per_pos'] = get_empty_tag_dict()
#             self.claim_dict['title']['ids'] = {}
            self.save_claim()
    
    def process_claims_selected(self, claim_dictionary):
        # add ids to selected dictionary which are the proof
        if 'ids_selected' not in self.claim_dict:
            interpreter_list = claim_dictionary.evidence
            id_list = []
            for interpreter in interpreter_list:
                for proof in interpreter:
                    title = proof[2]
                    if title is not None:
                        id = wiki_database.get_id_from_title(title)
                        id_list.append(id)
            self.claim_dict['ids_selected'] = id_list
        
        # === add from selected_ids in claim_dictionary === #    
        if 'docs_selected' in claim_dictionary:
            claim = Claim(claim_dictionary)
            self.claim_dict['ids_selected'] += claim['docs_selected']
            
        # === save === #
        self.save_claim()
        
    def process_claim(self, claim):
        self.claim_dict['claim']['text'] = claim
        self.save_claim()

    def process_tags(self, tag_list, n_gram):
#         self.claim_dict['claim'][str(n_gram) +'_gram'] = {}
        if n_gram == 1:
            self.claim_dict['claim'][str(n_gram) +'_gram']['tag_list'] = tag_list
        else:
            raise ValueError('written for n_gram == 1')
        self.save_claim()
    
    def process_tf_idf_experiment(self, tag_2_id_dict, tf_idf_db, mydict_ids, mydict_tf_idf):
        tf_idf_name = get_tf_idf_name(experiment_nr)
        if tf_idf_db.n_gram == 1:
            doc = tf_idf_db.vocab.wiki_database.nlp(self.claim_dict['claim']['text'])
            
            tag_list = [word.pos_ for word in doc]        
#             claim_doc_tokenizer = ClaimDocTokenizer(doc, tf_idf_db.vocab.delimiter_words)
#             n_grams_dict, nr_words = claim_doc_tokenizer.get_n_grams(tf_idf_db.vocab.method_tokenization, tf_idf_db.vocab.n_gram)
            # === write tf-idf values === #
            claim_text = Text(doc)
            tokenized_claim_list = claim_text.process(tf_idf_db.vocab.method_tokenization)
#             print(tag_list, tokenized_claim_list)
            idx = 0
            for i in range(len(tag_list)):
                tag = tag_list[i]
                word = tokenized_claim_list[i]
            
                pos_id = tag_2_id_dict[tag]
                
                with HiddenPrints():
                    dictionary = get_dict_from_n_gram([word], mydict_ids, mydict_tf_idf, tf_idf_db)
#                 print(len(dictionary))
                if len(dictionary) < 2000:
                    for id, tf_idf_value in dictionary.items():
                        # === create dictionary if does not exist === #
                        if str(id) not in self.claim_dict['title']['1_gram']:
                            self.claim_dict['title']['1_gram'][str(id)] = {}
                            title = tf_idf_db.vocab.wiki_database.get_title_from_id(id)

                            doc = tf_idf_db.vocab.wiki_database.nlp(title)
                            claim_doc_tokenizer = ClaimDocTokenizer(doc, tf_idf_db.vocab.delimiter_words)
                            n_grams_dict_title, nr_words_title = claim_doc_tokenizer.get_n_grams(tf_idf_db.vocab.method_tokenization, tf_idf_db.vocab.n_gram)

                            self.claim_dict['title']['1_gram'][str(id)]['nr_words'] = nr_words_title

                        if vocab.method_tokenization[0] not in self.claim_dict['title']['1_gram'][str(id)].keys():
                            self.claim_dict['title']['1_gram'][str(id)][vocab.method_tokenization[0]] = {}
                            if tf_idf_name not in self.claim_dict['title']['1_gram'][str(id)][vocab.method_tokenization[0]].keys():
                                self.claim_dict['title']['1_gram'][str(id)][vocab.method_tokenization[0]][tf_idf_name] = get_empty_tag_dict()

                        self.claim_dict['title']['1_gram'][str(id)][vocab.method_tokenization[0]][tf_idf_name][str(pos_id)] += tf_idf_value   
                idx += 1
            self.save_claim()
        else:
            raise ValueError('Adapt function for bigrams')
        
        self.save_claim()
    
    def process_nr_words_per_pos(self, tf_idf_db, tag_2_id_dict):
        if tf_idf_db.n_gram == 1:
            doc = tf_idf_db.vocab.wiki_database.nlp(self.claim_dict['claim']['text'])
            claim_doc_tokenizer = ClaimDocTokenizer(doc, tf_idf_db.vocab.delimiter_words)
            n_grams_dict, nr_words = claim_doc_tokenizer.get_n_grams(tf_idf_db.vocab.method_tokenization, tf_idf_db.vocab.n_gram)

            self.claim_dict['claim']['1_gram']['nr_words'] = sum(n_grams_dict.values())
            
            for key, count in n_grams_dict.items():
                tag, word = get_tag_word_from_wordtag(key, vocab.delimiter_tag_word)
                pos_id = tag_2_id_dict[tag]
                self.claim_dict['claim']['1_gram']['nr_words_per_pos'][str(pos_id)] += count
            self.save_claim()
        else:
            raise ValueError('Adapt function for bigrams')

    def save_claim(self):
        with HiddenPrints():
            dict_save_json(self.claim_dict, self.path_claim)

In [69]:
tag_2_id_dict = get_tag_2_id_dict()

experiment_nr = 37

with HiddenPrints():
    vocab, tf_idf_db = get_vocab_tf_idf_from_exp(experiment_nr)

doc = vocab.wiki_database.nlp(claim.claim)
claim_doc_tokenizer = ClaimDocTokenizer(doc, vocab.delimiter_words)
n_grams, nr_words = claim_doc_tokenizer.get_n_grams(vocab.method_tokenization, tf_idf_db.vocab.n_gram)

claim_dict = {}

# claim features
claim_dict['claim'] = {}
claim_dict['claim']['nr_words'] = sum(n_grams.values())
claim_dict['claim']['nr_words_per_pos'] = get_empty_tag_dict()

for key, count in n_grams.items():
    tag, word = get_tag_word_from_wordtag(key, vocab.delimiter_tag_word)
    pos_id = tag_2_id_dict[tag]
    claim_dict['claim']['nr_words_per_pos'][pos_id] += count
    



In [143]:
# title features
claim_dict['title'] = {}
claim_dict['title']['ids'] = {}
# get_empty_dict_claim()
# tf idf per tag
experiment_nr = 31
vocab, tf_idf_db = get_vocab_tf_idf_from_exp(experiment_nr)

doc = vocab.wiki_database.nlp(claim.claim)
claim_doc_tokenizer = ClaimDocTokenizer(doc, vocab.delimiter_words)
n_grams, nr_words = claim_doc_tokenizer.get_n_grams(vocab.method_tokenization, tf_idf_db.vocab.n_gram)

# === initialise databases === #
path_title_ids = tf_idf_db.path_ids_dict
path_title_tf_idf = tf_idf_db.path_tf_idf_dict
mydict_ids = SqliteDict(path_title_ids)
mydict_tf_idf = SqliteDict(path_title_tf_idf)

dictionary = get_dict_from_n_gram(mydict_ids, mydict_tf_idf, tf_idf_db)

tf_idf_name = get_tf_idf_name(experiment_nr)
tag_nr = get_tag_2_id_dict()

for id, tf_idf_value in dictionary.items():
    if id in claim_dict['title']['ids'].keys():
        claim_dict['title']['ids'][id][vocab.method_tokenization[0]][tf_idf_name][] += tf_idf_value
    else:
        claim_dict['title']['ids'][id] = get_empty_dict_claim()
        claim_dict['title']['ids'][id][vocab.method_tokenization[0]][tf_idf_name][] += tf_idf_value



SyntaxError: invalid syntax (<ipython-input-143-b597943da182>, line 22)

In [137]:
def get_dict_from_n_gram(mydict_ids, mydict_tf_idf, tf_idf_db):
    
    dictionary = {}

    for word in n_grams:
        try:
            word_id_list = mydict_ids[word].split(tf_idf_db.delimiter)[1:]
            word_tf_idf_list = mydict_tf_idf[word].split(tf_idf_db.delimiter)[1:]
        except KeyError:
            print('KeyError', word)
            word_id_list = []
            word_tf_idf_list = []
        for j in range(len(word_id_list)):
            id = int(word_id_list[j])       
            tf_idf = float(word_tf_idf_list[j])
            try:
                dictionary[id] = dictionary[id] + tf_idf
            except KeyError:
                dictionary[id] = tf_idf
    return dictionary

def get_empty_dict_claim():
    empty_dict = {}
    empty_dict['nr_words_title'] = None
    empty_dict['nr_words_per_pos'] = None
    empty_dict['tokenize'] = {}
    empty_dict['tokenize']['matches_per_pos'] = get_empty_tag_dict()
    empty_dict['tokenize']['tf_idf'] = get_empty_tag_dict()
    empty_dict['tokenize']['raw_count_idf'] = get_empty_tag_dict()
    empty_dict['tokenize']['idf'] = get_empty_tag_dict()
#     empty_dict['tokenize']['raw_count_sum_per_pos'] = get_empty_tag_dict()
#     empty_dict['tokenize']['max_sum_idf'] = 0
    empty_dict['tokenize_lemma'] = {}
    empty_dict['tokenize_lemma']['matches_per_pos'] = get_empty_tag_dict()
    empty_dict['tokenize_lemma']['tf_idf_sum_per_pos'] = get_empty_tag_dict()
    empty_dict['tokenize_lemma']['raw_count_idf_sum_per_pos'] = get_empty_tag_dict()
    empty_dict['tokenize_lemma']['idf_sum_per_pos'] = get_empty_tag_dict()
    empty_dict['tokenize_lemma']['raw_count_sum_per_pos'] = get_empty_tag_dict()
    empty_dict['tokenize_lemma']['max_sum_idf'] = 0
    return empty_dict
       
def get_empty_tag_dict():
    tag_2_id_dict = get_tag_2_id_dict()
    empty_tag_dict = {}
    for pos_id in range(len(tag_2_id_dict)):
        empty_tag_dict[str(pos_id)] = 0
    return empty_tag_dict

def get_tag_2_id_dict():
    tag_2_id_dict = {}
    tag_list = ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE']

    for i in range(len(tag_list)):
        tag = tag_list[i]
        tag_2_id_dict[tag] = i
    return tag_2_id_dict

In [29]:
def get_vocab_tf_idf_from_exp(experiment_nr):
    file_name = 'experiment_%.2d.json'%(experiment_nr)
    path_experiment = os.path.join(config.ROOT, config.CONFIG_DIR, file_name)

    with open(path_experiment) as json_data_file:
        data = json.load(json_data_file)

    vocab = VocabularySqlite(wiki_database = wiki_database, n_gram = data['n_gram'],
        method_tokenization = data['method_tokenization'], tags_in_db_flag = data['tags_in_db_flag'], 
        source = data['vocabulary_source'], tag_list_selected = data['tag_list_selected'])

    tf_idf_db = TFIDFDatabaseSqlite(vocabulary = vocab, method_tf = data['method_tf'], method_df = data['method_df'],
        delimiter = data['delimiter'], threshold = data['threshold'], source = data['tf_idf_source'])
    return vocab, tf_idf_db

In [37]:
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__


# print 'This will print'

# blockPrint()
# print "This won't"

# enablePrint()
# print "This will too"

  Using cached https://files.pythonhosted.org/packages/a1/5b/0fab3fa533229436533fb504bb62f4cf7ea29541a487a9d1a0749876fc23/spacy-2.1.4-cp36-cp36m-manylinux1_x86_64.whl
Requirement already up-to-date: requests<3.0.0,>=2.13.0 in /home/bmelman/C_disk/03_environment/03_fever/lib/python3.6/site-packages (from spacy)
Requirement already up-to-date: murmurhash<1.1.0,>=0.28.0 in /home/bmelman/C_disk/03_environment/03_fever/lib/python3.6/site-packages (from spacy)
  Using cached https://files.pythonhosted.org/packages/f4/c1/d76ccdd12c716be79162d934fe7de4ac8a318b9302864716dde940641a79/wasabi-0.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/34/46/b1d0bb71d308e820ed30316c5f0a017cb5ef5f4324bcbc7da3cf9d3b075c/blis-0.2.4-cp36-cp36m-manylinux1_x86_64.whl
Requirement already up-to-date: jsonschema<3.1.0,>=2.6.0 in /home/bmelman/C_disk/03_environment/03_fever/lib/python3.6/site-packages (from spacy)
  Using cached https://files.pythonhosted.org/packages/a9/f1/3df317939a07b2fc

# Experiment